In [1]:
import torch
import torchvision
from torchvision import transforms as T

from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

In [3]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 77.2MB/s]


In [4]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [5]:
!wget http://images.cocodataset.org/val2017/000000037777.jpg

--2024-09-02 20:43:55--  http://images.cocodataset.org/val2017/000000037777.jpg
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.19.125, 52.217.198.41, 52.216.240.212, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.19.125|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40833 (40K) [image/jpeg]
Saving to: ‘000000037777.jpg’

000000037777.jpg    100%[===================>]  39.88K  --.-KB/s    in 0.002s  

2024-09-02 20:43:55 (18.7 MB/s) - ‘000000037777.jpg’ saved [40833/40833]



In [7]:
image = Image.open("/content/000000037777.jpg")

In [8]:
transform = T.ToTensor()
image_tensor = transform(image)

In [9]:
image_tensor

tensor([[[0.6784, 0.6588, 0.6510,  ..., 0.6118, 0.6039, 0.6157],
         [0.6784, 0.6627, 0.6549,  ..., 0.6078, 0.6000, 0.6118],
         [0.6784, 0.6627, 0.6588,  ..., 0.6039, 0.5961, 0.6078],
         ...,
         [0.6431, 0.6196, 0.6039,  ..., 0.6000, 0.6039, 0.6118],
         [0.6549, 0.6314, 0.6118,  ..., 0.5922, 0.5961, 0.6039],
         [0.6549, 0.6353, 0.6235,  ..., 0.5882, 0.5922, 0.6000]],

        [[0.6863, 0.6667, 0.6588,  ..., 0.6196, 0.6118, 0.6235],
         [0.6863, 0.6706, 0.6627,  ..., 0.6157, 0.6078, 0.6196],
         [0.6863, 0.6706, 0.6667,  ..., 0.6118, 0.6039, 0.6157],
         ...,
         [0.5961, 0.5725, 0.5569,  ..., 0.6157, 0.6196, 0.6275],
         [0.6078, 0.5843, 0.5647,  ..., 0.6078, 0.6118, 0.6196],
         [0.6078, 0.5882, 0.5765,  ..., 0.6039, 0.6078, 0.6157]],

        [[0.6745, 0.6549, 0.6471,  ..., 0.6078, 0.6000, 0.6118],
         [0.6745, 0.6588, 0.6510,  ..., 0.6039, 0.5961, 0.6078],
         [0.6745, 0.6588, 0.6549,  ..., 0.6000, 0.5922, 0.

In [10]:
with torch.no_grad():
  prediction = model([image_tensor])

Retorna todos os objetos detectados na imagem que foram treinados na ResNet50.

No dicionário, há bounding box(xi,yi,xf,yf), labels e a probabilidade de cada objeto

In [14]:
prediction

[{'boxes': tensor([[291.8007,  75.4907, 351.0379, 229.7333],
          [137.4995, 126.4641, 196.7451, 192.7782],
          [233.8998, 187.4761, 257.3713, 206.0314],
          [ 90.1806, 183.4493, 288.7284, 227.4105],
          [215.5326, 186.7111, 232.5018, 201.2200],
          [229.1946, 178.4547, 242.4028, 189.1366],
          [204.3889, 187.5528, 220.3058, 207.7724],
          [231.4346, 200.6802, 247.8696, 217.5509],
          [217.1698, 200.3502, 231.5009, 214.5559],
          [204.0359, 181.5956, 255.1724, 215.4856],
          [ 98.9669, 120.1149, 110.5809, 174.5164],
          [102.2467, 119.0014, 109.8358, 135.8731],
          [264.9863, 133.9124, 294.1249, 137.5284],
          [232.7792, 188.8490, 246.7510, 200.8903],
          [256.4788, 132.5554, 295.4839, 138.3106],
          [175.0202, 110.8700, 185.6927, 137.3661],
          [273.3569, 135.1965, 294.8690, 137.8414],
          [263.4073, 128.6209, 298.7946, 137.3732],
          [197.2355, 181.2733, 263.8224, 219.3763],
   